# 第五次课后练习 之二（选做）

在本次作业中，如果你在配置环境中遇到了问题，可以参考助教的解决方案：

首先我们在将这个文件放在一个文件夹中（可以和这门课程的其他作业放在一起，但是别放在桌面这种一大堆其他文件的文件夹里）。

随后运行：
```
python3 -m venv myenv
source myenv/bin/activate
pip install openai
pip install requests
pip install socksio
```

然后在 notebook 中切换 ipykernel 为 myenv（如果你采用的是 VSCode 的话，可以看看右上角的 ipykernel，点一下就可以切换了），就可以运行了。

如果你发现还存在包缺失的情况，注意采用 Restart，不然环境可能同步不过来。

**负责助教：吴迪**

<span style="color:red; font-weight:bold;">请将作业文件命名为 第五次课后练习-之二+姓名+学号.ipynb, 例如 第五次课后练习-之二+张三+1000000000.ipynb</span>

#  请认真阅读代码，理解学习代码的功能

## **0.1** 读取网页内容，调用大语言模型API进行中文摘要
    

In [ ]:
import re
import requests
from openai import OpenAI

# 目标URL列表
target_urls = [
    "https://arxiv.org/abs/2410.03761",
    "https://arxiv.org/abs/2305.15186"
]

# OpenRouter API配置，这个是提供免费deepseek服务的站点，
# 也可以替换成直接访问deepseekAPI的版本
API_ENDPOINT = "https://openrouter.ai/api/v1"
API_KEY = "<YOUR_API_KEY>"

# 正则表达式，用于解析网页摘要（读取html网页对象的content字段内容）
pattern = r'<meta\s+property="og:description"\s+content="(.*?)"\s*/>'

# 初始化OpenAI客户端
client = OpenAI(
    base_url=API_ENDPOINT,
    api_key=API_KEY,
)

def generate_summary(text: str) -> str:
    """生成文本摘要"""
    try:
        # 调用DeepSeek API生成摘要
        # 如果你采用了其他的站点的内容，你可能需要修改这里的 model 的内容。
        completion = client.chat.completions.create(
            model="deepseek/deepseek-chat:free",
            messages=[
                {
                    "role": "system",
                    "content": "Please write a 300 word(character) summary of the user's text in Chinese"
                },
                {
                    "role": "user",
                    "content": text
                }
            ]
        )
        # 返回生成的摘要内容
        return completion.choices[0].message.content
    except Exception as e:
        print(f"Error generating summary: {e}")
        return "Failed to generate summary"

def fetch_webpage_content(url: str) -> str:
    """获取网页内容"""
    try:
        # 发送HTTP GET请求获取网页内容
        response = requests.get(url, timeout=10)  # 设置超时时间
        response.raise_for_status()  # 检查请求是否成功
        return response.text
    except requests.exceptions.RequestException as e:
        print(f"Error fetching {url}: {e}")
        return None

def parse_summary(html_content: str) -> str:
    """从网页内容中解析出摘要"""
    try:
        # 使用正则表达式从HTML中提取摘要，读取html网页对象的content字段
        match = re.search(pattern, html_content)
        if match:
            return match.group(1)
        else:
            raise Exception("No summary found in the webpage")
    except Exception as e:
        print(f"Error parsing summary: {e}")
        return None

def main():
    # 遍历目标URL列表
    
    docs = ""
    
    for url in target_urls:
        print(f"Processing URL: {url}")
        
        # 获取网页内容
        html_content = fetch_webpage_content(url)
        if not html_content:
            print('access error')
            continue  # 如果获取失败，跳过当前URL
        
        # 解析摘要
        summary = parse_summary(html_content)
        if not summary:
            print('parse error')
            continue  # 如果解析失败，跳过当前URL
            
        docs += summary
        
        # 生成中文摘要
        chinese_summary = generate_summary(summary)
        
        # 打印结果
        print(f"Original Summary: {summary}")
        print(f"Chinese Summary: {chinese_summary}\n")

    chinese_summary = generate_summary(docs)
    print(f"Original message: {docs}")
    print(f"multi doc Summary: {chinese_summary}\n")
if __name__ == "__main__":
    main()

# 1 仿照上面的框架，实现一个自己版本的多文档（或多轮对话）的分析功能。可以用更复杂的爬虫获得需要的信息。也可以直接从本地读取数据文件实现有实际意义的功能。

完成有新意有创意工作的同学，或在作业过程中觉得有心得或者自己拓展学习到有价值内容的，可以在文件名最后加一个#号。例如第五次课后练习+张三+1000000000+#.ipynb
只是完成学习的同学，没有尝试改进探索工作的可以不提交这个作业